# **Cuaderno 3: Agregaciones para Business Intelligence**

### **TEMA:** "De GROUP BY a GROUP BY CUBE: Reportes Multidimensionales en una Sola Consulta"

**Objetivo:** Los participantes comprenderán las limitaciones de `UNION ALL` para crear reportes con subtotales y aprenderán a usar `ROLLUP`, `CUBE` y `GROUPING SETS` para generar agregaciones multidimensionales de forma eficiente y elegante.

In [0]:
# Celda 1: Script para generar y configurar el conjunto de datos del taller
from pyspark.sql import SparkSession
from pyspark.sql.types import DecimalType # Importante: Importar el tipo de dato necesario
from faker import Faker
import random
from datetime import datetime, timedelta

# --- SEMILLA PARA REPRODUCIBILIDAD ---
# Garantiza que los datos generados sean siempre los mismos.
SEED = 2025
Faker.seed(SEED)
random.seed(SEED)

# Inicializar Faker
fake = Faker('es_ES')

# --- FUNCIONES DE GENERACIÓN DE DATOS ---
def generar_usuarios(n=1000):
    """Genera una lista de diccionarios de usuarios."""
    return [{'id_usuario': 1000 + i, 'nombre': fake.name(), 'email': fake.email(), 'fecha_registro': fake.date_time_between(start_date='-2y'), 'ciudad': fake.city()} for i in range(n)]

def generar_productos(n=500):
    """Genera una lista de productos."""
    categorias = ['Electrónica', 'Hogar', 'Ropa', 'Libros', 'Deportes', 'Juguetes', 'Alimentos']
    data = []
    for i in range(n):
        data.append({
            'id_producto': 2000 + i,
            'nombre_producto': fake.word().capitalize() + " " + fake.word(),
            'categoria': random.choice(categorias),
            'precio_unitario': round(random.uniform(5.0, 350.0), 2)
        })
    return data

def generar_pedidos(usuarios, productos, n=10000):
    """Genera una lista de pedidos, vinculando usuarios y productos."""
    data = []
    for i in range(n):
        usuario = random.choice(usuarios)
        producto = random.choice(productos)
        cantidad = random.randint(1, 5)
        data.append({
            'id_pedido': 3000 + i,
            'id_usuario': usuario['id_usuario'],
            'id_producto': producto['id_producto'],
            'cantidad': cantidad,
            'monto': round(cantidad * producto['precio_unitario'], 2),
            'fecha_pedido': fake.date_time_between(start_date=usuario['fecha_registro'])
        })
    return data

# --- CREACIÓN DE DATAFRAMES ---
print("Generando datos simulados...")
usuarios_data = generar_usuarios()
productos_data = generar_productos()
pedidos_data = generar_pedidos(usuarios_data, productos_data)

usuarios_df = spark.createDataFrame(usuarios_data)
productos_df = spark.createDataFrame(productos_data)
pedidos_df = spark.createDataFrame(pedidos_data)

# --- CREACIÓN DE BASE DE DATOS Y TABLAS SILVER ---
db_name = "curso_arquitecturas"
spark.sql(f"CREATE DATABASE IF NOT EXISTS {db_name}")
spark.sql(f"USE {db_name}")

print(f"Usando la base de datos: '{db_name}'")

# Guardar usuarios
usuarios_df.selectExpr("id_usuario", "nombre as nombre_usuario", "email", "fecha_registro", "ciudad") \
    .write.format("delta").mode("overwrite").option("overwriteSchema", "true").saveAsTable("usuarios_silver")

# ==================== LA CORRECCIÓN ESTÁ AQUÍ ====================
# 1. Definimos explícitamente el tipo de dato para 'precio_unitario' antes de escribir.
productos_df_casteado = productos_df.withColumn("precio_unitario", productos_df["precio_unitario"].cast(DecimalType(10, 2)))

# 2. Escribimos el DataFrame corregido, permitiendo la sobreescritura del esquema para robustez.
productos_df_casteado.write.format("delta").mode("overwrite").option("overwriteSchema", "true").saveAsTable("productos_silver")
# =================================================================

# Convertimos el DataFrame de pedidos a una tabla SQL temporal para poder usar funciones SQL
pedidos_df.createOrReplaceTempView("pedidos_temp")

# Guardar pedidos con el esquema bien definido usando SQL
spark.sql("""
CREATE OR REPLACE TABLE pedidos_silver AS
SELECT 
  id_pedido, 
  id_usuario, 
  id_producto, 
  cantidad, 
  CAST(monto AS DECIMAL(18, 2)) AS monto_total, 
  fecha_pedido AS ts_pedido,
  TO_DATE(fecha_pedido) as id_fecha 
FROM pedidos_temp
""")

# --- CREACIÓN DE DIM_FECHA ---
spark.sql("""
CREATE OR REPLACE TABLE dim_fecha (
  id_fecha DATE NOT NULL, anio INT, mes INT, dia INT, trimestre INT,
  nombre_mes STRING, nombre_dia_semana STRING, tipo_dia STRING
);
""")
spark.sql("""
INSERT INTO dim_fecha
SELECT
  fecha AS id_fecha, YEAR(fecha) AS anio, MONTH(fecha) AS mes, DAY(fecha) AS dia,
  QUARTER(fecha) AS trimestre, DATE_FORMAT(fecha, 'MMMM') AS nombre_mes,
  DATE_FORMAT(fecha, 'EEEE') AS nombre_dia_semana,
  CASE WHEN DAYOFWEEK(fecha) IN (1, 7) THEN 'Fin de Semana' ELSE 'Día de Semana' END AS tipo_dia
FROM (SELECT EXPLODE(SEQUENCE(TO_DATE('2022-01-01'), TO_DATE('2026-12-31'), INTERVAL 1 DAY)) AS fecha);
""")

print("Tablas Silver y dim_fecha creadas/actualizadas exitosamente.")
print("\n¡Entorno listo para el Módulo 2!")

**Contexto:** El equipo de Business Intelligence necesita un único reporte que resuma el total de ventas (`monto_total`) con múltiples niveles de granularidad para un análisis multidimensional:

1.  El detalle de ventas por **Categoría y Año** combinados.
2.  Un **Subtotal** consolidado para cada **Categoría** (sumando todos los años).
3.  Un **Subtotal** consolidado para cada **Año** (sumando todas las categorías).
4.  Un **Gran Total** de todas las ventas.

**Análisis:** La primera aproximación, y la más común para quienes no conocen las mejoras de `GROUP BY`, es calcular cada nivel de agregación por separado y luego juntar los resultados con `UNION ALL`.

Este enfoque es un **anti-patrón** por dos razones principales:
* **Rendimiento Pobre:** Obliga al motor de Spark a escanear la tabla de hechos (`pedidos_silver`) **cuatro veces**, una por cada `SELECT`. En tablas con miles de millones de filas, esto es computacionalmente inaceptable.
* **Mantenimiento Complejo:** El código es repetitivo y difícil de mantener. Si se necesita añadir un nuevo filtro (ej. `WHERE ciudad = 'Bogotá D.C.'`), hay que recordarlo y aplicarlo en cuatro lugares distintos, lo que es propenso a errores.

In [0]:
%sql
-- "Mala" Solución: Unir 4 consultas separadas
-- Consulta 1: Detalle por Categoría y Año
SELECT
  pr.categoria,
  d.anio,
  SUM(p.monto_total) AS total_ventas
FROM pedidos_silver p
JOIN productos_silver pr ON p.id_producto = pr.id_producto
JOIN dim_fecha d ON p.id_fecha = d.id_fecha
GROUP BY pr.categoria, d.anio

UNION ALL

-- Consulta 2: Subtotal por Categoría
SELECT
  pr.categoria,
  NULL AS anio,
  SUM(p.monto_total) AS total_ventas
FROM pedidos_silver p
JOIN productos_silver pr ON p.id_producto = pr.id_producto
JOIN dim_fecha d ON p.id_fecha = d.id_fecha
GROUP BY pr.categoria

UNION ALL

-- Consulta 3: Subtotal por Año
SELECT
  NULL AS categoria,
  d.anio,
  SUM(p.monto_total) AS total_ventas
FROM pedidos_silver p
JOIN productos_silver pr ON p.id_producto = pr.id_producto
JOIN dim_fecha d ON p.id_fecha = d.id_fecha
GROUP BY d.anio

UNION ALL

-- Consulta 4: Gran Total
SELECT
  NULL AS categoria,
  NULL AS anio,
  SUM(p.monto_total) AS total_ventas
FROM pedidos_silver p
JOIN productos_silver pr ON p.id_producto = pr.id_producto
JOIN dim_fecha d ON p.id_fecha = d.id_fecha;

**Análisis:** `ROLLUP` es una extensión de `GROUP BY` que calcula subtotales de manera jerárquica. Es perfecto para crear reportes que necesitan un "desglose" (drill-down) desde un nivel general a uno más detallado, siguiendo el orden de las columnas especificadas.

`ROLLUP(A, B)` calculará las agregaciones para:
1.  `(A, B)`
2.  `(A)`
3.  `()` (El gran total)

In [0]:
%sql
-- Con ROLLUP, obtenemos 3 de los 4 niveles que necesitamos
SELECT
  pr.categoria,
  d.anio,
  SUM(p.monto_total) AS total_ventas
FROM pedidos_silver p
JOIN productos_silver pr ON p.id_producto = pr.id_producto
JOIN dim_fecha d ON p.id_fecha = d.id_fecha
GROUP BY ROLLUP(pr.categoria, d.anio)
ORDER BY pr.categoria, d.anio;

**`GROUP BY ROLLUP(categoria, anio)`** le dice a la base de datos:

1.  Primero, agrupa por la combinación de `(categoria, anio)`.
2.  Luego, "sube un nivel" y agrupa solo por `(categoria)`, creando un subtotal para cada una.
3.  Finalmente, "sube al nivel más alto" y calcula el gran total para `()`.

Las filas de subtotal se identifican porque la columna que se está agregando en ese nivel aparece como `NULL`.

**Limitación:** `ROLLUP` es jerárquico. `ROLLUP(categoria, anio)` **no** nos da el subtotal solo por `anio`. Para eso, necesitamos una herramienta más poderosa.

**Análisis:** `CUBE` es la herramienta definitiva para análisis multidimensional. A diferencia de `ROLLUP`, `CUBE` genera subtotales para **todas las combinaciones posibles** de las columnas especificadas.

`CUBE(A, B)` calculará las agregaciones para:
1.  `(A, B)`
2.  `(A)`
3.  `(B)`
4.  `()` (El gran total)

Esto resuelve nuestro problema de negocio de forma completa y eficiente, escaneando los datos **una sola vez**.

In [0]:
%sql
-- Con CUBE, obtenemos los 4 niveles requeridos en una sola consulta
SELECT
  pr.categoria,
  d.anio,
  SUM(p.monto_total) AS total_ventas
FROM pedidos_silver p
JOIN productos_silver pr ON p.id_producto = pr.id_producto
JOIN dim_fecha d ON p.id_fecha = d.id_fecha
GROUP BY CUBE(pr.categoria, d.anio)
ORDER BY pr.categoria, d.anio;

**Análisis:** `CUBE` crea un "cubo" de datos con todas las posibles vistas de agregación. Pero, ¿cómo distinguimos una fila de subtotal (`NULL`) de una fila donde el valor original era realmente `NULL`?

La respuesta es la función `grouping_id()`. Esta función devuelve un entero que actúa como un mapa de bits, indicando qué columnas están siendo agregadas en esa fila.

Para `CUBE(categoria, anio)`:
- `grouping_id() = 0`: Agregación por `(categoria, anio)` (ninguna es `NULL` de subtotal).
- `grouping_id() = 1`: Agregación por `(categoria)` (`anio` es `NULL` de subtotal).
- `grouping_id() = 2`: Agregación por `(anio)` (`categoria` es `NULL` de subtotal).
- `grouping_id() = 3`: Gran total (`categoria` y `anio` son `NULL` de subtotal).

In [0]:
%sql
-- Usando grouping_id() para crear una columna descriptiva
SELECT
  pr.categoria,
  d.anio,
  grouping_id() AS nivel_agregacion,
  CASE grouping_id()
    WHEN 0 THEN 'Detalle por Categoría y Año'
    WHEN 1 THEN 'Subtotal por Categoría'
    WHEN 2 THEN 'Subtotal por Año'
    WHEN 3 THEN 'Gran Total'
  END AS descripcion_nivel,
  SUM(p.monto_total) AS total_ventas
FROM pedidos_silver p
JOIN productos_silver pr ON p.id_producto = pr.id_producto
JOIN dim_fecha d ON p.id_fecha = d.id_fecha
GROUP BY CUBE(pr.categoria, d.anio)
ORDER BY pr.categoria, d.anio;

**Análisis:** ¿Qué pasa si no necesitas *todas* las combinaciones de `CUBE`? `GROUPING SETS` te da el control quirúrgico para especificar exactamente qué combinaciones de agregación quieres, evitando cálculos innecesarios.

`ROLLUP` y `CUBE` son, en realidad, atajos para `GROUPING SETS` más complejos.

In [0]:
%sql
-- Requerimiento: Solo necesitamos el detalle (categoria, anio) y el subtotal por (anio)
SELECT
  pr.categoria,
  d.anio,
  SUM(p.monto_total) AS total_ventas
FROM pedidos_silver p
JOIN productos_silver pr ON p.id_producto = pr.id_producto
JOIN dim_fecha d ON p.id_fecha = d.id_fecha
GROUP BY GROUPING SETS (
  (pr.categoria, d.anio),  -- Set 1: El nivel de detalle
  (d.anio)                 -- Set 2: El subtotal por año
)
ORDER BY
  pr.categoria, d.anio;

**Tu Tarea:** El equipo de marketing quiere analizar la adquisición de nuevos usuarios. Necesitan un único reporte que muestre, usando la tabla `usuarios_silver` y la `dim_fecha`:

**Requerimiento:** Generar el conteo de nuevos usuarios (`COUNT(id_usuario)`) para los siguientes niveles:
1.  El número de usuarios registrados por **año**.
2.  El número de usuarios registrados por **ciudad**.
3.  El número de usuarios registrados por **año y ciudad**.
4.  El **total general** de usuarios registrados.

**Pregunta:** ¿Qué operador (`ROLLUP` o `CUBE`) es el más adecuado para generar todos estos niveles en una sola consulta? Escribe la consulta completa.

In [0]:
%sql
-- Escribe tu solución para el ejercicio aquí...

---
**Ideas Clave:**
* **Anti-Patrón:** Usar múltiples `UNION ALL` para crear reportes con subtotales es ineficiente y difícil de mantener.
* **`ROLLUP`:** Genera subtotales **jerárquicos**. Ideal para desgloses (drill-down).
* **`CUBE`:** Genera subtotales para **todas las combinaciones**. Perfecto para exploración multidimensional.
* **`GROUPING SETS`:** Ofrece **control total** para especificar solo las agregaciones que necesitas.
* **`grouping_id()`:** Es la función esencial para identificar y etiquetar los niveles de subtotal en tus resultados, haciéndolos legibles para humanos y herramientas de BI.

**Lección Principal:** Abandona el `UNION ALL` para subtotales y adopta las mejoras de `GROUP BY`. Escribirás código más limpio, más rápido y más fácil de mantener.

In [0]:
%sql
-- Solución: CUBE es la herramienta correcta porque necesitamos todas las combinaciones posibles.
SELECT
  d.anio AS anio_registro,
  u.ciudad,
  CASE grouping_id()
      WHEN 0 THEN 'Detalle por Año y Ciudad'
      WHEN 1 THEN 'Subtotal por Año'
      WHEN 2 THEN 'Subtotal por Ciudad'
      WHEN 3 THEN 'Gran Total'
  END AS descripcion_nivel,
  COUNT(u.id_usuario) AS total_usuarios
FROM
  usuarios_silver u
JOIN 
  dim_fecha d ON TO_DATE(u.fecha_registro) = d.id_fecha
GROUP BY CUBE(d.anio, u.ciudad)
ORDER BY
  anio_registro, ciudad;